In [23]:
import time
import cv2
import math
import random

In [32]:
def generate_dict_of_pixeles_per_meter(number_of_cameras: int) -> dict:
    dictionary = {}
    
    for i in range(number_of_cameras):
        dictionary[i] = round(random.uniform(5, 15), 1)
    
    return dictionary

In [34]:
pixel_per_meter_for_each_camera = generate_dict_of_pixeles_per_meter(10)

WIDTH = 1280
HEIGHT = 720

RECTANGLE_COLOR = (0, 255, 0)
RECTANGLE_WIDTH = 3

In [22]:
class MainLoop:
    def __init__(self, path_to_cascade: str, path_to_video: str, camera_id: int):
        self.cars = []
        self.camera_id = camera_id

        self.start(path_to_cascade, path_to_video)
    
    def _delete_trackers_by_quality(self, frame):
        """Методом update мы проверяем качество обекта в трекере и если оно ниже 5, удаляем трекер."""
        
        if self.cars:
            self.cars = [car for car in self.cars if car.tracker.update(frame) < 5]
            
            
    def _find_car_by_coordinates(self, x, y, w, h):
        """Проверяет создали ли уже tracker для этого автомобиля, то есть не детектим ли одни и те же объекты."""
        
        horizontal_center = x + 0.5 * w
        vertical_center = y + 0.5 * h
        
        for car in self.cars:
            tracked_position = car.tracker.get_position()

            t_x = int(tracked_position.left())
            t_y = int(tracked_position.top())
            t_w = int(tracked_position.width())
            t_h = int(tracked_position.height())

            t_x_bar = t_x + 0.5 * t_w
            t_y_bar = t_y + 0.5 * t_h

            if ((t_x <= horizontal_center <= (t_x + t_w)) and 
                (t_y <= vertical_center <= (t_y + t_h)) and 
                (x <= t_x_bar <= (x + w)) and 
                (y <= t_y_bar <= (y + h))
               ):
                return car
            
        return None
    
    
    def _create_new_tracker(self, x, y, w, h):
        """Создает новый трекер для автомобиля."""
        
        pixel_per_meter = pixel_per_meter_for_each_camera[self.camera_id]
        tracker = dlib.correlation_tracker()
        tracker.start_track(frame, dlib.rectangle(x, y, x + w, y + h))

        new_car = Automobile(pixel_per_meter, tracker, [x, y, w, h])
        self.cars.append(new_car)

        
    def start(self, path_to_cascade, path_to_video):
        """Цикл обработки кадров видео."""
        
        car_cascade = cv2.CascadeClassifier(path_to_cascade)
        video = cv2.VideoCapture(path_to_video)
        
        while True:
            start_time = time.time()
            flag, frame = self.video.read()
        
            if not flag:
                break

            frame = cv2.resize(frame, (WIDTH, HEIGHT))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cars = car_cascade.detectMultiScale(gray, scaleFactor=1.05, minSize=(24, 24))
            
            self.delete_trackers_by_quality(frame)
            
            for (x, y, w, h) in cars:
                match_car_id = self._find_car_by_coordinates(x, y, w, h)
                        
                if match_car_id is None:
                    self._create_new_car_tracker(x, y, w, h)
                
                for car in self.cars:
                    car.draw_rectangle(start_time)

                    if car.start_coord != car.end_coord:
                        car.start_coord = car.end_coord
                        
                        if car.speed is None:
                            car.calculate_distance()



In [35]:
class Automobile:
    def __init__(self, pixel_per_meter=None, tracker=None, start_coord=None, end_coord=None):
        self.tracker = tracker
        self.start_coord = start_coord
        self.end_coord = end_coord
        
        self.pixel_per_meter = pixel_per_meter
        self.speed = None
        self.fps = 0
    
    def calculate_distance(self):
        """Высчитывает евклидово расстояние, переводит пикисили в метры и считает скорость в километрах в час."""
        
        d_pixels = math.sqrt(math.pow(self.end_coord[0] - self.start_coord[0], 2) +
                             math.pow(self.end_coord[1] - self.start_coord[1], 2))

        d_meters = d_pixels / self.pixel_per_meter
        
        # умножаем на 3, переводим в километры в час
        self.speed = d_meters * self.fps * 3
        
    
    def draw_rectangle(self, frame, start_time):
        """Рисует оконтовку квадратом вокруг объекта."""
        
        tracked_position = self.tracker.get_position()

        t_x = int(tracked_position.left())
        t_y = int(tracked_position.top())
        t_w = int(tracked_position.width())
        t_h = int(tracked_position.height())

        cv2.rectangle(frame, (t_x, t_y), (t_x + t_w, t_y + t_h), RECTANGLE_COLOR, RECTANGLE_WIDTH)

        self.end_coord = [t_x, t_y, t_w, t_h]
        
        self._find_pixel_per_meter(start_time)
        
    
    def _find_frame_rate(self, start_time):
        """Считает частоту кадров."""
        
        end_time = time.time()

        if not (end_time == start_time):
            self.fps = 1.0 / (end_time - start_time)
        
        